In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib, pickle
import gensim
from gensim.models import word2vec


In [2]:
data = pd.read_csv('spotify_songs.csv')
data.head()

,track_id,track_name,track_artist,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,playlist_genre,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,6f807x0ima9a1j3VPbc7VN,I Don't Care (with Justin Bieber) - Loud Luxur...,Ed Sheeran,66,2oCs0DGTsRO98Gh5ZSl2Cx,I Don't Care (with Justin Bieber) [Loud Luxury...,2019-06-14,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,6,-2.634,1,0.0583,0.1020,0.000000,0.0653,0.518,122.036,194754
1,0r7CVbZTWZgbTCYdfa2P31,Memories - Dillon Francis Remix,Maroon 5,67,63rPSO264uRjW1X5E6cWv6,Memories (Dillon Francis Remix),2019-12-13,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,11,-4.969,1,0.0373,0.0724,0.004210,0.3570,0.693,99.972,162600
2,1z1Hg7Vb0AhHDiEmnDE79l,All the Time - Don Diablo Remix,Zara Larsson,70,1HoSmj2eLcsrR0vE9gThr4,All the Time (Don Diablo Remix),2019-07-05,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,1,-3.432,0,0.0742,0.0794,0.000023,0.1100,0.613,124.008,176616
3,75FpbthrwQmzHlBJLuGdC7,Call You Mine - Keanu Silva Remix,The Chainsmokers,60,1nqYsOef1yKKuGOVchbsk6,Call You Mine - The Remixes,2019-07-19,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,7,-3.778,1,0.1020,0.0287,0.000009,0.2040,0.277,121.956,169093
4,1e8PAfcKUYoKkxPhrHqw4x,Someone You Loved - Future Humans Remix,Lewis Capaldi,69,7m7vv9wlQ4i0LFuJiE2zsQ,Someone You Loved (Future Humans Remix),2019-03-05,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,1,-4.672,1,0.0359,0.0803,0.000000,0.0833,0.725,123.976,189052


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32833 entries, 0 to 32832
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   track_id                  32833 non-null  object 
 1   track_name                32828 non-null  object 
 2   track_artist              32828 non-null  object 
 3   track_popularity          32833 non-null  int64  
 4   track_album_id            32833 non-null  object 
 5   track_album_name          32828 non-null  object 
 6   track_album_release_date  32833 non-null  object 
 7   playlist_name             32833 non-null  object 
 8   playlist_id               32833 non-null  object 
 9   playlist_genre            32833 non-null  object 
 10  playlist_subgenre         32833 non-null  object 
 11  danceability              32833 non-null  float64
 12  energy                    32833 non-null  float64
 13  key                       32833 non-null  int64  
 14  loudne

In [4]:
data.duplicated().sum()

0

In [5]:
df_selected = data[['track_artist', 'track_name']]
df_selected.shape

(32833, 2)

### Modèle de système de recommandation spotify avec Word2Vec

##### Introduction
L'application d'écoute musicale spotify est utilisée par en majorité 90% de la population pour se distraire, d'etre de bon humeur et d'etre motivé. A cet effet quand nous recherchons une album ou song sur cette appli, il se peut qu'on constate des albumns ou chansons similaires à celle écouter. Il faut donc comprendre que celle ci utilise un système de recommandations qui sur le calcul de la distance euclidienne entre les differentes topics afin de ressortir les similarités entre les artistes names ou albumns préféreés. C'est un modèle d'intelligence Artificielle qui est à la base des suggestions que nous constatons et observons. 

#### Technique 

Pour mettre en place ce genre de système , plusieurs techniques sont utilisées comme : L'apprentissage automatique ou le Deep Learning ou les Techniques de Traitement de Langage Naturel (PNL) qui fait l'objet d'actualité ces derniers moments. 

Nous utiliserons dans ce tutoriel, une technique simple : un modèle pré-entrainé, Word2Vec, utilisé en NLP pour la classification textuelles et détection de sentiment. 

In [6]:
df_selected.to_csv('lifestring.csv', index = False)

In [7]:
dt = pd.read_csv('lifestring.csv')
dt.head()

,track_artist,track_name
0,Ed Sheeran,I Don't Care (with Justin Bieber) - Loud Luxur...
1,Maroon 5,Memories - Dillon Francis Remix
2,Zara Larsson,All the Time - Don Diablo Remix
3,The Chainsmokers,Call You Mine - Keanu Silva Remix
4,Lewis Capaldi,Someone You Loved - Future Humans Remix


In [8]:
### model Word2vec
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess, simple_tokenize


song = dt.groupby('track_name')['track_artist'].apply(list)
song
model = Word2Vec(sentences = song, vector_size= 200, workers= 4, 
                 window= 5, alpha= 0.025, 
                 min_count=5)


In [9]:
model.build_vocab(song)

In [10]:
model.wv.most_similar('Ed Sheeran')

[('T-Pain', 0.2250695377588272),
 ('Getter', 0.19596436619758606),
 ('Musiq Soulchild', 0.19372694194316864),
 ('The Cranberries', 0.19169889390468597),
 ('Arrested Youth', 0.18472768366336823),
 ('Lost Frequencies', 0.18436619639396667),
 ('Lil Peep', 0.18370260298252106),
 ('Charlie Puth', 0.17775875329971313),
 ('Giaime', 0.17679323256015778),
 ('In This Moment', 0.1733146458864212)]

In [11]:
### vectors d'embedding
vector = model.wv['Ed Sheeran']
print(vector)

[ 4.32189414e-03  1.69439614e-03 -1.63053290e-03 -3.93099803e-03
 -1.29127654e-03  6.86922576e-04  3.31688789e-03  1.84051204e-03
  4.29834326e-04  1.71254203e-03 -2.58630048e-03  3.88272270e-03
  1.35713851e-03  3.05075594e-03  2.63234996e-03  2.20517977e-03
  4.26452095e-03  3.61026032e-03 -1.79397431e-03  4.47924063e-03
 -4.59798519e-03 -4.88244416e-03 -3.97266122e-03  1.67822558e-03
  7.20505777e-04  1.49827870e-03  2.40048533e-03 -3.37661244e-03
 -2.14101583e-05 -6.29297458e-04  4.84718336e-03  5.05203661e-03
 -1.72580767e-04 -9.83904582e-04 -1.10837992e-03 -1.96098164e-03
 -2.12615170e-03  4.14161524e-03  4.36899299e-03  4.54403739e-03
 -3.03283479e-04  2.64614820e-03  4.23867488e-03 -2.61042127e-03
 -4.83507756e-03 -4.12332779e-03  2.61892774e-03  4.08272026e-03
 -3.41707189e-03 -3.51579464e-03 -1.61550590e-03 -2.29658815e-03
  3.54893529e-03 -3.57805798e-03  3.21767037e-03 -5.73740865e-04
  4.99370042e-03 -1.23530207e-03 -4.93444130e-03  1.69779488e-03
 -1.94905943e-03 -1.34250

In [12]:
similar_songs = model.wv.most_similar('Ed Sheeran', topn = 5)
print("Le nom des artistes similaires sont: ", similar_songs)

Le nom des artistes similaires sont:  [('T-Pain', 0.2250695377588272), ('Getter', 0.19596436619758606), ('Musiq Soulchild', 0.19372694194316864), ('The Cranberries', 0.19169889390468597), ('Arrested Youth', 0.18472768366336823)]


In [13]:
model.save('recommandations.model')

In [14]:
modeling = Word2Vec.load('recommandations.model')

In [15]:
modeling.wv.most_similar('Getter')

[('Faith Evans', 0.2266390025615692),
 ('Bea Miller', 0.20767760276794434),
 ('Lil Tecca', 0.20128928124904633),
 ('John Mellencamp', 0.1966869831085205),
 ('Ed Sheeran', 0.19596438109874725),
 ('Montrose', 0.1913846731185913),
 ('8Ball', 0.17415890097618103),
 ('Bryan Adams', 0.17360085248947144),
 ('Stormzy', 0.17139051854610443),
 ('The Black Keys', 0.16753362119197845)]